In [2]:
from model_prediction import combine_roots_and_themes
import pandas as pd

# Read the CSV file into a DataFrame
input = pd.read_csv('sentences_author.csv')
input = input.sample(n=20, random_state=42)
print(input)

           author                                           sentence
0   Reviewer eS3u  This work proposes LSTNet, a self-supervised m...
5   Reviewer eS3u  The performance of aligned shape pairs under t...
33  Reviewer wiS9                                    See weaknesses.
13  Reviewer jP4i                  3) Experimental results are good.
19  Reviewer wiS9  This paper introduces LSTNet, which leverages ...
50  Reviewer Frem  2. The proposed method is straightforward and ...
36  Reviewer a6Ps  The SO(3)-invariant network design intrinsical...
26  Reviewer wiS9  2.2.2 Second, the so-called local shape transf...
44  Reviewer a6Ps  2. Will the network still be functional if the...
12  Reviewer jP4i  2) The idea of factorizing point cloud descrip...
54  Reviewer Frem  4. The whole method is mainly built upon the e...
3   Reviewer eS3u  The self- and cross-reconstruction training st...
34  Reviewer a6Ps  This paper attempts to register point cloud pr...
30  Reviewer wiS9  4. For the SO(3

In [ ]:
results = combine_roots_and_themes(input)
results

Some layers from the model checkpoint at /home/nana/DASP_report_template/backend/models/attitude_root/ were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /home/nana/DASP_report_template/backend/models/attitude_root/ and are newly initialized: ['dropout_819']
You should probably TRAIN this model on a down-stream task to be able to use it for prediction

,Attitude_roots,Frequency,Descriptions,Comments
8,Substance(MET),0.6,none,"[[Reviewer eS3u, [2. The proposed method is st..."
0,Clarity(MET),0.4,none,"[[Reviewer eS3u, [The reason why other methods..."
7,Substance(EXP),0.4,none,"[[Reviewer eS3u, [3) Experimental results are ..."
1,Clarity(RWK),0.2,none,"[[Reviewer eS3u, [4. For the SO(3)-equivariant..."
2,Meaningful-comparison(MET),0.2,none,"[[Reviewer eS3u, [4) Please compare the propos..."
4,Originality(MET),0.2,none,"[[Reviewer eS3u, [4. The whole method is mainl..."
3,Meaningful-comparison(RWK),0.2,none,"[[Reviewer eS3u, [4) Please compare the propos..."
6,Substance(DAT),0.2,none,"[[Reviewer eS3u, [2. The proposed method is st..."
5,Originality(PDI),0.2,none,"[[Reviewer eS3u, [2) The idea of factorizing p..."


: 

In [7]:
results.to_csv('attitude_roots_themes.csv', index=False)

In [7]:
def create_clusters(row):
    # Extract root and themes
    root = row["attitude_root"]
    themes = row["attitude_themes"]
    
    # Combine root with each theme
    clusters = [f"{root}({theme})" for theme in themes]
    return clusters

df = results
df.loc[:, "clusters"] = df.apply(create_clusters, axis=1)
df = df.explode("clusters", ignore_index=True)

In [8]:
df

,author,sentence,attitude_root_number,attitude_root,attitude_themes,clusters
0,Reviewer jP4i,3) Experimental results are good.,7,Substance,"[EXP, RES]",Substance(EXP)
1,Reviewer jP4i,3) Experimental results are good.,7,Substance,"[EXP, RES]",Substance(RES)
2,Reviewer Frem,2. The proposed method is straightforward and ...,7,Substance,"[DAT, MET]",Substance(DAT)
3,Reviewer Frem,2. The proposed method is straightforward and ...,7,Substance,"[DAT, MET]",Substance(MET)
4,Reviewer wiS9,"2.2.2 Second, the so-called local shape transf...",7,Substance,"[EXP, MET]",Substance(EXP)
5,Reviewer wiS9,"2.2.2 Second, the so-called local shape transf...",7,Substance,"[EXP, MET]",Substance(MET)
6,Reviewer a6Ps,2. Will the network still be functional if the...,7,Substance,[MET],Substance(MET)
7,Reviewer jP4i,2) The idea of factorizing point cloud descrip...,4,Originality,"[MET, PDI]",Originality(MET)
8,Reviewer jP4i,2) The idea of factorizing point cloud descrip...,4,Originality,"[MET, PDI]",Originality(PDI)
9,Reviewer Frem,4. The whole method is mainly built upon the e...,4,Originality,[MET],Originality(MET)


In [14]:
# Group by 'author' and 'clusters', and aggregate the sentences into a list
aggregated_df = df.groupby(['clusters', 'author'])['sentence'].apply(list).reset_index()
aggregated_df

,clusters,author,sentence
0,Clarity(MET),Reviewer eS3u,[The reason why other methods are much better ...
1,Clarity(MET),Reviewer wiS9,[4. For the SO(3)-equivariant and -invariant m...
2,Meaningful-comparison(MET),Reviewer jP4i,[4) Please compare the proposed method with mo...
3,Meaningful-comparison(RWK),Reviewer jP4i,[4) Please compare the proposed method with mo...
4,Originality(MET),Reviewer Frem,[4. The whole method is mainly built upon the ...
5,Originality(MET),Reviewer jP4i,[2) The idea of factorizing point cloud descri...
6,Originality(PDI),Reviewer jP4i,[2) The idea of factorizing point cloud descri...
7,Substance(DAT),Reviewer Frem,[2. The proposed method is straightforward and...
8,Substance(EXP),Reviewer jP4i,[3) Experimental results are good.]
9,Substance(EXP),Reviewer wiS9,"[2.2.2 Second, the so-called local shape trans..."


In [16]:
final_df = aggregated_df.groupby('clusters').agg(
            comments=('sentence', lambda x: [[author, sentences] for author, sentences in zip(aggregated_df['author'], x)])
        ).reset_index()

In [17]:
final_df

,clusters,comments
0,Clarity(MET),"[[Reviewer eS3u, [The reason why other methods..."
1,Meaningful-comparison(MET),"[[Reviewer eS3u, [4) Please compare the propos..."
2,Meaningful-comparison(RWK),"[[Reviewer eS3u, [4) Please compare the propos..."
3,Originality(MET),"[[Reviewer eS3u, [4. The whole method is mainl..."
4,Originality(PDI),"[[Reviewer eS3u, [2) The idea of factorizing p..."
5,Substance(DAT),"[[Reviewer eS3u, [2. The proposed method is st..."
6,Substance(EXP),"[[Reviewer eS3u, [3) Experimental results are ..."
7,Substance(MET),"[[Reviewer eS3u, [2. The proposed method is st..."
8,Substance(RES),"[[Reviewer eS3u, [3) Experimental results are ..."
